In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls drive/MyDrive/PhD_USP_Adailton/Tese/4_paper_cluster_labeling/Results/

B004LOMB2Q_run_10_.xls	       com.spotify.music_run_1_.xls
B004LOMB2Q_run_1_.xls	       com.spotify.music_run_2_.xls
B004LOMB2Q_run_2_.xls	       com.spotify.music_run_3_.xls
B004LOMB2Q_run_3_.xls	       com.spotify.music_run_4_.xls
B004LOMB2Q_run_4_.xls	       com.spotify.music_run_5_.xls
B004LOMB2Q_run_5_.xls	       com.spotify.music_run_6_.xls
B004LOMB2Q_run_6_.xls	       com.spotify.music_run_7_.xls
B004LOMB2Q_run_7_.xls	       com.spotify.music_run_8_.xls
B004LOMB2Q_run_8_.xls	       com.spotify.music_run_9_.xls
B004LOMB2Q_run_9_.xls	       com.twitter.android_run_10_.xls
B004SIIBGU_run_10_.xls	       com.twitter.android_run_2_.xls
B004SIIBGU_run_1_.xls	       com.twitter.android_run_3_.xls
B004SIIBGU_run_2_.xls	       com.twitter.android_run_4_.xls
B004SIIBGU_run_3_.xls	       com.twitter.android_run_5_.xls
B004SIIBGU_run_4_.xls	       com.twitter.android_run_6_.xls
B004SIIBGU_run_5_.xls	       com.twitter.android_run_7_.xls
B004SIIBGU_run_6_.xls	       com.twitter.android_run_8_

In [3]:
path='drive/MyDrive/PhD_USP_Adailton/Tese/4_paper_cluster_labeling/Results/'

In [4]:
!git clone https://github.com/jsdabrowski/CAiSE-20

Cloning into 'CAiSE-20'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [5]:
import pandas as pd

df_data = pd.read_excel('CAiSE-20/Ground_truth.xlsx')
df_data

,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated)
0,com.zentertain.photoeditor,gp:AOqpTOEW40L9WXqCjzq04bqaZImgMdzlczxIF3_ibs8...,1,May be i can check,NaN,NaN,NaN,NaN
1,com.zentertain.photoeditor,gp:AOqpTOF57AQPvmnCiWYurwLY-F2-mej25ON8RAFk-Ls...,1,It make me happy,NaN,NaN,NaN,NaN
2,com.zentertain.photoeditor,gp:AOqpTOHYdmt72q4tSD8TZ8A5fZQqGivlBkIbWuHuJMZ...,1,I have a normal phone and it made 1 of my pics...,NaN,pics,NaN,pics
3,com.zentertain.photoeditor,gp:AOqpTOFYnXMShrDJPS0jpM04pFQxYOJN1LDuX3lSNm0...,1,Love it so much,NaN,NaN,NaN,NaN
4,com.zentertain.photoeditor,gp:AOqpTOF_JO496wnThQ2kcYlPct_g1GhOmQyyVvHp4VV...,1,Cant get to install,NaN,install,NaN,install
...,...,...,...,...,...,...,...,...
2057,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,1,Every time I go on I can't browse or do anythi...,NaN,NaN,NaN,NaN
2058,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,2,I am fully connected to wifi and offline mode ...,NaN,offline mode,NaN,offline mode
2059,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,3,"Loved this app when it worked, can't use at al...",NaN,NaN,NaN,NaN
2060,com.spotify.music,gp:AOqpTOE-vqukBoo4GbnnTJnBesSlsJR9w2yGydeMlIK...,4,Will cancel if not fixed very soon...,NaN,NaN,NaN,NaN


In [6]:
!pip install git+https://github.com/rmarcacini/sentence-transformers
!pip install gdown
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!unzip distiluse-base-multilingual-cased.zip -d language_model

  Cloning https://github.com/rmarcacini/sentence-transformers to /tmp/pip-req-build-8xbtkbt6
  Running command git clone -q https://github.com/rmarcacini/sentence-transformers /tmp/pip-req-build-8xbtkbt6
     |████████████████████████████████| 890kB 3.9MB/s 
     |████████████████████████████████| 901kB 7.9MB/s 
     |████████████████████████████████| 1.2MB 10.7MB/s 
     |████████████████████████████████| 3.0MB 17.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp37-none-any.whl size=101874 sha256=81c6cb214b8d675e5344da3a26f7df484eeac12f8c09392eb0232ef299fad305
  Stored in directory: /tmp/pip-ephem-wheel-cache-_wmchdrw/wheels/88/3c/66/55ee9fb698480d5a5116a8257c15dc363323e4922fb8ad361b
Successfully built sentence-transformers
Downloading...
From: https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
To: /content/distiluse-base-multilingual-cased.zip
504MB [00:03, 156MB/s]
Archive:  distiluse-base-multilingual-cased.zip
   creating: lang

In [7]:
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

np.set_printoptions(threshold=100)
logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO,handlers=[LoggingHandler()])

language_model = SentenceTransformer('./language_model')

2021-06-01 15:01:35 - Load pretrained SentenceTransformer: ./language_model
2021-06-01 15:01:35 - Load SentenceTransformer from folder: ./language_model
2021-06-01 15:01:38 - Use pytorch device: cpu


In [8]:
pd.DataFrame(df_data['App id'].unique())

,0
0,com.zentertain.photoeditor
1,B004LOMB2Q
2,B004SIIBGU
3,com.whatsapp
4,B005ZXWMUS
5,com.twitter.android
6,B0094BB4TW
7,com.spotify.music


In [9]:
APP_NAME = pd.DataFrame(df_data['App id'].unique())[0][4]
RUN=3

In [10]:
sentence_embedding_cache = {}
requirement_embedding_cache = {}
L_sentence = []
L_requirement = []
for index,row in df_data.iterrows():
  L_sentence.append(row['Sentence content'])
  try:
    for r in row['Feature (All Annotated)'].split(';'):
      #print(r.strip())
      L_requirement.append(r.strip())
  except:
    1

L_sentence_embedding = list(language_model.encode(L_sentence))
L_requirement_embedding = list(language_model.encode(L_requirement))



In [11]:
for i in range(0,len(L_sentence)):
  sentence_embedding_cache[L_sentence[i]] = L_sentence_embedding[i]

for i in range(0,len(L_requirement)):
  requirement_embedding_cache[L_requirement[i]] = L_requirement_embedding[i]

In [12]:
df_app = df_data[df_data['App id']==APP_NAME]
L = []
for index,row in df_app.iterrows():
  L.append(sentence_embedding_cache[row['Sentence content']])
df_app['sentence_embedding'] = L
df_app

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated),sentence_embedding
984,B005ZXWMUS,AZFD9X1QH49M4,1,if you don't have this in your library for app...,NaN,NaN,NaN,NaN,"[0.013342008, -0.018081037, 0.02233074, -0.020..."
985,B005ZXWMUS,AZFD9X1QH49M4,2,To be honest I would tell everybody I know to ...,NaN,movie,NaN,movie,"[-0.02837871, -0.01996382, -0.006706098, 0.008..."
986,B005ZXWMUS,A1F66F1M3B2OKI,1,out of no where the video will freeze up for a...,NaN,video,NaN,video,"[-0.030757956, -0.0100252945, 0.04964122, -0.0..."
987,B005ZXWMUS,A1F66F1M3B2OKI,2,I have to go through that a couple of times bu...,NaN,NaN,NaN,NaN,"[0.009794, -0.019251859, 0.03229382, -0.059420..."
988,B005ZXWMUS,A2AVUADUI8PHVP,1,awesome app but first you have to log out and ...,NaN,log out;log in,NaN,log out;log in,"[0.044009726, 0.0001370064, -0.008946776, -0.0..."
...,...,...,...,...,...,...,...,...,...
1320,B005ZXWMUS,A2RK3QEUEFRNAN,5,The only fix I've found is to watch subtitled ...,NaN,watch subtitled films,NaN,watch subtitled films,"[0.03226805, 0.011807728, -0.007928347, 0.0167..."
1321,B005ZXWMUS,A2RK3QEUEFRNAN,6,Works great and the selection of Netflix is st...,selection,NaN,NaN,selection,"[0.024893597, 0.0032350898, -0.017533725, -0.0..."
1322,B005ZXWMUS,AQZ3Z5QIW2WC7,1,This was a smart purchase that has proven to b...,free trial,movies,NaN,free trial;movies,"[0.00018239394, 0.027448721, 0.04815843, -0.00..."
1323,B005ZXWMUS,AQZ3Z5QIW2WC7,2,I'm pleasantly surprised.,NaN,NaN,NaN,NaN,"[-0.031237576, 0.0005915034, -0.03338208, -0.0..."


In [13]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples
import numpy as np
import math

def get_kmeans(df_):

  df_temp = df_.copy()
  X = np.array(df_temp['requirement_embedding'].to_list())
  num_clusters = int(math.sqrt(len(df_temp)))
  if num_clusters < 2:
    print('Error! Dataset size!')
    return
  if num_clusters >= 30:
    num_clusters = 30

  cluster_parent = df_temp['cluster'].unique()[0]
  print('Running cluster from ',cluster_parent,' total=',len(X),' num_clusters=',num_clusters)
  kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=RUN, init='random',verbose=0,batch_size=1000,max_no_improvement=1000).fit(X)
  df_temp['cluster_parent'] = cluster_parent
  df_temp['cluster'] = np.array(kmeans.labels_)+(10*cluster_parent)
  print('Cluster parent',df_temp['cluster_parent'].unique())
  print('Cluster ',df_temp['cluster'].unique())

  return df_temp

In [14]:
requirements = set()
df_temp = df_data[df_data['App id']==APP_NAME]
for index,row in df_temp[['Feature (All Annotated)']].dropna().iterrows():
    for r in row['Feature (All Annotated)'].split(';'):
      requirements.add(r.strip())

df_requirements = pd.DataFrame(requirements)
df_requirements.columns =  ['requirement']
L = []
for r in requirements:
    L.append(requirement_embedding_cache[r])
df_requirements['requirement_embedding'] = L
df_requirements

2021-06-01 15:03:25 - NumExpr defaulting to 2 threads.


,requirement,requirement_embedding
0,watching Supernatural,"[-0.008343686, 0.02932607, 0.0076835672, 0.048..."
1,watch what they like,"[0.0052363225, -0.014871125, -0.02393147, 0.01..."
2,"options to choose between TV shows,movies","[-0.017233405, 0.007912642, 0.009400168, -0.01..."
3,stream certain tv shows,"[-0.04656879, -0.0019365641, -0.00591911, -0.0..."
4,television,"[-0.07427363, 0.042894084, -0.013909168, -0.03..."
...,...,...
196,netflix movies,"[-0.051189914, 0.023808898, -0.038188208, 0.04..."
197,watch,"[0.026891626, 0.03702663, -0.044766076, 0.0216..."
198,watch appropriate television,"[-0.053592447, 0.0058764215, 0.0028492433, 0.0..."
199,OLD SHOWS,"[0.018474484, 0.04304514, 0.013055836, 0.03081..."


In [15]:
stack = []
df_temp = df_requirements.copy()
df_temp['cluster_parent'] = 0
df_temp['cluster'] = 1
stack.append(df_temp)

hc = []

while(len(stack)!=0):
  df_temp = stack.pop()
  if len(df_temp) <= 20: continue
  df_temp = get_kmeans(df_temp)
  hc.append(df_temp)
  for cluster_ in df_temp.cluster.unique():
    stack.append(df_temp[df_temp.cluster==cluster_])

Running cluster from  1  total= 201  num_clusters= 14
Cluster parent [1]
Cluster  [20 19 11 17 14 23 21 12 18 16 22 10 13 15]
Running cluster from  12  total= 44  num_clusters= 6
Cluster parent [12]
Cluster  [120 125 121 123 122 124]
Running cluster from  17  total= 42  num_clusters= 6
Cluster parent [17]
Cluster  [172 175 174 171 173 170]
Running cluster from  11  total= 23  num_clusters= 4
Cluster parent [11]
Cluster  [112 110 113 111]
Running cluster from  20  total= 22  num_clusters= 4
Cluster parent [20]
Cluster  [201 203 202 200]


In [16]:
df_hc = pd.concat(hc)
df_hc

,requirement,requirement_embedding,cluster_parent,cluster
0,watching Supernatural,"[-0.008343686, 0.02932607, 0.0076835672, 0.048...",1,20
1,watch what they like,"[0.0052363225, -0.014871125, -0.02393147, 0.01...",1,19
2,"options to choose between TV shows,movies","[-0.017233405, 0.007912642, 0.009400168, -0.01...",1,20
3,stream certain tv shows,"[-0.04656879, -0.0019365641, -0.00591911, -0.0...",1,20
4,television,"[-0.07427363, 0.042894084, -0.013909168, -0.03...",1,20
...,...,...,...,...
166,TV show selections,"[-0.029234808, 0.07071592, 0.032474667, 0.0450...",20,203
167,TV series,"[-0.09690852, 0.018585173, 0.024346963, -0.000...",20,201
172,movie best seletion of TV show,"[-0.043976992, -0.0039587067, 0.06633475, 0.02...",20,201
182,catch up on an old television series,"[-0.05629269, -0.027166069, 0.020622138, -0.01...",20,201


# Centroid Labeling (Baseline)

In [17]:
def centroid_labeling(df_hc):

  cluster_labels = {}

  df_hc1 = df_hc.copy() 
  for cluster in df_hc1.cluster.unique():
    cluster_centroid = np.mean(np.array(df_hc1[df_hc1.cluster==cluster].requirement_embedding.to_list()),axis=0)
    min_dist = 999999999
    cluster_label = None

    for index,row in df_hc1[df_hc1.cluster==cluster].iterrows():
      requirement_emb = row['requirement_embedding']
      dist = np.linalg.norm(requirement_emb-cluster_centroid)
      if dist < min_dist:
        cluster_label = row['requirement']
        min_dist = dist
    cluster_labels[cluster]=cluster_label

  L = []
  for index,row in df_hc1.iterrows():
    L.append(cluster_labels[row['cluster']])

  df_hc1['cluster_label'] = L
  return df_hc1

df_hc1 = centroid_labeling(df_hc)
df_hc1

,requirement,requirement_embedding,cluster_parent,cluster,cluster_label
0,watching Supernatural,"[-0.008343686, 0.02932607, 0.0076835672, 0.048...",1,20,watch tv shows
1,watch what they like,"[0.0052363225, -0.014871125, -0.02393147, 0.01...",1,19,watch what I want
2,"options to choose between TV shows,movies","[-0.017233405, 0.007912642, 0.009400168, -0.01...",1,20,watch tv shows
3,stream certain tv shows,"[-0.04656879, -0.0019365641, -0.00591911, -0.0...",1,20,watch tv shows
4,television,"[-0.07427363, 0.042894084, -0.013909168, -0.03...",1,20,watch tv shows
...,...,...,...,...,...
166,TV show selections,"[-0.029234808, 0.07071592, 0.032474667, 0.0450...",20,203,choices for TV shows
167,TV series,"[-0.09690852, 0.018585173, 0.024346963, -0.000...",20,201,watch tv shows
172,movie best seletion of TV show,"[-0.043976992, -0.0039587067, 0.06633475, 0.02...",20,201,watch tv shows
182,catch up on an old television series,"[-0.05629269, -0.027166069, 0.020622138, -0.01...",20,201,watch tv shows


# Avaliação Baseline

In [18]:
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics import f1_score
from sklearn.metrics.cluster import normalized_mutual_info_score



def isNaN(num):
  return num != num


def eval_labeling(df_hc_,df_app):

  
  results = []

  for cluster_parent in df_hc_.cluster_parent.unique():

    L = []

    for index,row in df_app.iterrows():
      for r in df_hc_[df_hc_.cluster_parent==cluster_parent].requirement.unique():
        if isNaN(row['Feature (All Annotated)'])==False:
          if r in row['Feature (All Annotated)']:
            L.append(row.to_dict())

    df_app2 = pd.DataFrame(L)

    cluster_reference = []
    cluster_labeling = []
    
    for index,row in df_app2.iterrows():

      ####### REFERENCE CLUSTER #######
      cluster_sel = -1
      min_dist = 999999999
      sentence_embedding = row['sentence_embedding']

      df_temp = df_hc_[df_hc_.cluster_parent==cluster_parent]
      for cluster in df_temp.cluster.unique():
        cluster_centroid = np.mean(np.array(df_temp[df_temp.cluster==cluster].requirement_embedding.to_list()),axis=0)
        dist = np.linalg.norm(sentence_embedding-cluster_centroid)
        if dist < min_dist:
          cluster_sel = cluster
          min_dist = dist
      
      cluster_reference.append(cluster_sel)


      ####### LABELING CLUSTER #######
      cluster_sel = -1
      min_dist = 999999999
      sentence_embedding = row['sentence_embedding']

      df_temp = df_hc_[df_hc_.cluster_parent==cluster_parent]
      for cluster in df_temp.cluster.unique():
        cluster_centroid = requirement_embedding_cache[df_temp[df_temp.cluster==cluster].head(1).cluster_label.values[0]]
        dist = np.linalg.norm(sentence_embedding-cluster_centroid)
        if dist < min_dist:
          cluster_sel = cluster
          min_dist = dist
      
      cluster_labeling.append(cluster_sel)

    # print(cluster_parent)
    # print(cluster_reference)
    # print(cluster_labeling)
    # print(v_measure_score(cluster_reference,cluster_labeling))
    # print(f1_score(cluster_reference,cluster_labeling,average='micro'))
    # print('--------------')
    results.append((cluster_parent,len(cluster_reference),v_measure_score(cluster_reference,cluster_labeling),f1_score(cluster_reference,cluster_labeling,average='micro')))

  return results

In [19]:
results_baseline = eval_labeling(df_hc1,df_app)
print(results_baseline)

[(1, 508, 0.49350652604135425, 0.452755905511811), (12, 86, 0.5042903113662981, 0.7906976744186046), (17, 150, 0.45524497423224164, 0.66), (11, 145, 0.566488583808825, 0.8896551724137931), (20, 33, 0.7572007474328185, 0.8484848484848486)]


# Proposed Cluster Labeling

In [20]:
df_app = df_data[df_data['App id']==APP_NAME]
L = []
for index,row in df_app.iterrows():
  L.append(sentence_embedding_cache[row['Sentence content']])
df_app['sentence_embedding'] = L
df_app

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,App id,Review id,Sentence id,Sentence content,Feature (Positive),Feature (Neutral),Feature (Negative),Feature (All Annotated),sentence_embedding
984,B005ZXWMUS,AZFD9X1QH49M4,1,if you don't have this in your library for app...,NaN,NaN,NaN,NaN,"[0.013342008, -0.018081037, 0.02233074, -0.020..."
985,B005ZXWMUS,AZFD9X1QH49M4,2,To be honest I would tell everybody I know to ...,NaN,movie,NaN,movie,"[-0.02837871, -0.01996382, -0.006706098, 0.008..."
986,B005ZXWMUS,A1F66F1M3B2OKI,1,out of no where the video will freeze up for a...,NaN,video,NaN,video,"[-0.030757956, -0.0100252945, 0.04964122, -0.0..."
987,B005ZXWMUS,A1F66F1M3B2OKI,2,I have to go through that a couple of times bu...,NaN,NaN,NaN,NaN,"[0.009794, -0.019251859, 0.03229382, -0.059420..."
988,B005ZXWMUS,A2AVUADUI8PHVP,1,awesome app but first you have to log out and ...,NaN,log out;log in,NaN,log out;log in,"[0.044009726, 0.0001370064, -0.008946776, -0.0..."
...,...,...,...,...,...,...,...,...,...
1320,B005ZXWMUS,A2RK3QEUEFRNAN,5,The only fix I've found is to watch subtitled ...,NaN,watch subtitled films,NaN,watch subtitled films,"[0.03226805, 0.011807728, -0.007928347, 0.0167..."
1321,B005ZXWMUS,A2RK3QEUEFRNAN,6,Works great and the selection of Netflix is st...,selection,NaN,NaN,selection,"[0.024893597, 0.0032350898, -0.017533725, -0.0..."
1322,B005ZXWMUS,AQZ3Z5QIW2WC7,1,This was a smart purchase that has proven to b...,free trial,movies,NaN,free trial;movies,"[0.00018239394, 0.027448721, 0.04815843, -0.00..."
1323,B005ZXWMUS,AQZ3Z5QIW2WC7,2,I'm pleasantly surprised.,NaN,NaN,NaN,NaN,"[-0.031237576, 0.0005915034, -0.03338208, -0.0..."


In [21]:
def get_contextual_embedding(requirement,df_app,alpha=0.9,show_progress_bar=False):

  df_feature = df_app[['Sentence content','Feature (All Annotated)','sentence_embedding']].dropna()
  df_req_temp = df_feature[df_feature['Feature (All Annotated)'].str.contains(requirement)]
  context_center = np.mean(np.array(df_req_temp['sentence_embedding'].to_list()),axis=0)
  requirement_emb = requirement_embedding_cache[requirement]
  contextual_embedding = (1.0-alpha)*requirement_emb + (alpha)*context_center

  return contextual_embedding

#get_contextual_embedding('premium',df_app,alpha=0.5)

In [22]:
def contextual_cluster_labeling(df_hc,df_app,alpha=0.1):

  cluster_labels = {}

  df_hc2 = df_hc.copy() 
  for cluster in df_hc2.cluster.unique():
    cluster_centroid = np.mean(np.array(df_hc2[df_hc2.cluster==cluster].requirement_embedding.to_list()),axis=0)
    min_dist = 999999999
    cluster_label = None

    for index,row in df_hc2[df_hc2.cluster==cluster].iterrows():
      requirement_emb = get_contextual_embedding(row['requirement'],df_app,alpha=alpha)
      #requirement_emb = row['requirement_embedding']
      dist = np.linalg.norm(requirement_emb-cluster_centroid)
      if dist < min_dist:
        cluster_label = row['requirement']
        min_dist = dist
    cluster_labels[cluster]=cluster_label

  L = []
  for index,row in df_hc2.iterrows():
    L.append(cluster_labels[row['cluster']])

  df_hc2['cluster_label'] = L
  return df_hc2



In [23]:
from tqdm.notebook import tqdm
results_proposal = {}
for alpha in tqdm(np.arange (0, 1.02, 0.02)):
  df_hc2 = contextual_cluster_labeling(df_hc,df_app,alpha=alpha)
  results_proposal[alpha] = eval_labeling(df_hc2,df_app)

# Salvando resultados

In [24]:
res1 = pd.DataFrame(results_baseline)
res1['method'] = 'baseline'
res1.columns = ['hie_level','total','v_measure','f1','method']


L = []
for alpha in results_proposal:
  for item in results_proposal[alpha]:
    R = list(item)
    R.append(alpha)
    L.append(R)



res2 = pd.DataFrame(L)
res2['method'] = 'proposal'
res2.columns = ['hie_level','total','v_measure','f1','alpha','method']

res_final = pd.concat([res1,res2])
#res_final.columns = ['hie_level','total','v_measure','f1','alpha','method']


res_final

,hie_level,total,v_measure,f1,method,alpha
0,1,508,0.493507,0.452756,baseline,NaN
1,12,86,0.504290,0.790698,baseline,NaN
2,17,150,0.455245,0.660000,baseline,NaN
3,11,145,0.566489,0.889655,baseline,NaN
4,20,33,0.757201,0.848485,baseline,NaN
...,...,...,...,...,...,...
250,1,508,0.366356,0.291339,proposal,1.0
251,12,86,0.401289,0.674419,proposal,1.0
252,17,150,0.404690,0.506667,proposal,1.0
253,11,145,0.246259,0.800000,proposal,1.0


In [25]:
res_final.to_excel(path+APP_NAME+'_run_'+str(RUN)+'_.xls')

In [26]:
res_final.groupby(['hie_level','method']).max()

total  v_measure        f1  alpha
hie_level method                                     
1         baseline    508   0.493507  0.452756    NaN
          proposal    508   0.502016  0.452756    1.0
11        baseline    145   0.566489  0.889655    NaN
          proposal    145   0.566489  0.889655    1.0
12        baseline     86   0.504290  0.790698    NaN
          proposal     86   0.504290  0.790698    1.0
17        baseline    150   0.455245  0.660000    NaN
          proposal    150   0.571429  0.713333    1.0
20        baseline     33   0.757201  0.848485    NaN
          proposal     33   0.757201  0.848485    1.0